# Transfer Learning

### Import Libraries

In [1]:

import pathlib
import os
import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential
from sklearn.model_selection import KFold


## Train with MonkeyPox Dataset

### Hyperparameters

In [2]:
data_root = pathlib.Path("../data/Augmented_Images")    # points to the folder containing the images that will be used for training

batch_size = 32
img_height = 224
img_width = 224


# Load dataset without splitting
dataset = tf.keras.utils.image_dataset_from_directory(
    data_root,
    image_size=(img_height, img_width),
    batch_size=batch_size,
    shuffle=True
)

class_names = np.array(dataset.class_names)
num_classes = len(class_names)

# Convert the dataset to a list of (image, label) pairs
image_paths, labels = [], []
for image_batch, label_batch in dataset:
    image_paths.extend(image_batch.numpy())
    labels.extend(label_batch.numpy())

image_paths = np.array(image_paths)
labels = np.array(labels)

Found 3192 files belonging to 2 classes.


## Training 

### Metrics

In [3]:
from sklearn.metrics import confusion_matrix, classification_report, recall_score, f1_score, ConfusionMatrixDisplay

# plot and save confusion matrix
def save_confusion_matrix(true_labels, predicted_labels, class_names, save_path):
    cm = confusion_matrix(true_labels, predicted_labels)
    disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=class_names)
    disp.plot(cmap=plt.cm.Blues)
    plt.title("Confusion Matrix")
    plt.savefig(save_path)
    plt.close()

# plot and save loss curves
def save_loss_curve(history, save_path):
    plt.figure(figsize=(10, 6))
    plt.plot(history['loss'], label='Training Loss', color='blue')
    plt.plot(history['val_loss'], label='Validation Loss', color='orange')
    plt.title("Training and Validation Loss Over Epochs")
    plt.xlabel("Epochs")
    plt.ylabel("Loss")
    plt.legend()
    plt.grid(True)
    plt.savefig(save_path)
    plt.close()

# compute and plot evaluation metrics (accuracy, sensitivity, specificity, F1 score)
def save_evaluation_metrics(true_labels, predicted_labels, history, cm, save_path):
    accuracy = history['val_accuracy'][-1]
    sensitivity = recall_score(true_labels, predicted_labels, average='macro')
    specificity = np.mean(np.diag(cm) / (np.diag(cm) + np.sum(cm, axis=0) - np.diag(cm)))
    f1 = f1_score(true_labels, predicted_labels, average='macro')

    metrics = {
        "Accuracy": accuracy,
        "Sensitivity (Recall)": sensitivity,
        "Specificity": specificity,
        "F1-Score": f1
    }

    plt.figure(figsize=(10, 6))
    plt.bar(metrics.keys(), metrics.values(), color=['darkturquoise', 'sandybrown', 'hotpink', 'limegreen'])
    plt.title("Model Evaluation Metrics")
    plt.ylim([0, 1])
    plt.yticks(np.arange(0, 1.1, 0.1))
    plt.ylabel("Score")
    plt.savefig(save_path)
    plt.close()
    return metrics

# save classification report
def save_classification_report(true_labels, predicted_labels, class_names, save_path):
    class_report = classification_report(true_labels, predicted_labels, target_names=class_names, digits=4)
    with open(save_path, "w") as f:
        f.write(class_report)

### Create and compile model

In [4]:
from sklearn.metrics import precision_score, recall_score, f1_score, roc_auc_score

# Function to create and compile the model
def create_model(num_classes, config):
    base_model = tf.keras.applications.MobileNetV2(
        input_shape=(img_height, img_width, 3),
        include_top=False,
        weights='imagenet'
    )
    base_model.trainable = False  # Freeze the base model

    model = Sequential([
        base_model,
        layers.GlobalAveragePooling2D(),
        layers.Dense(num_classes)
    ])

    
    # Select optimizer based on configuration
    if config["optimizer"] == "adam":
        optimizer = tf.keras.optimizers.Adam(learning_rate=config["learning_rate"])
    elif config["optimizer"] == "sgd":
        optimizer = tf.keras.optimizers.SGD(learning_rate=config["learning_rate"])
    else:
        raise ValueError(f"Unsupported optimizer: {config['optimizer']}")

    model.compile(
        optimizer=optimizer,
        loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
        metrics=['accuracy']
    )
    
    return model

# Function to calculate metrics
def calculate_metrics(true_labels, predictions):
    accuracy = np.mean(np.argmax(predictions, axis=1) == true_labels)
    precision = precision_score(true_labels, np.argmax(predictions, axis=1), average='macro')
    recall = recall_score(true_labels, np.argmax(predictions, axis=1), average='macro')
    f1 = f1_score(true_labels, np.argmax(predictions, axis=1), average='macro')
    auc = roc_auc_score(tf.keras.utils.to_categorical(true_labels), predictions, multi_class='ovr')
    return accuracy, precision, recall, f1, auc

# Function to save metrics, loss curve, and confusion matrix for the best model
def save_best_model_visuals(history, model, val_ds, class_names, weights_path, fold):
    # Generate predictions for the validation set
    val_predictions = model.predict(val_ds)
    val_predicted_ids = np.argmax(val_predictions, axis=-1)
    true_labels = np.concatenate([y for x, y in val_ds], axis=0)

    # Confusion Matrix
    confusion_matrix_path = os.path.join(weights_path, f"confusion_matrix_fold_{fold}.png")
    save_confusion_matrix(true_labels, val_predicted_ids, class_names, confusion_matrix_path)

    # Loss curve
    loss_curve_path = os.path.join(weights_path, f"loss_curve_fold_{fold}.png")
    save_loss_curve(history.history, loss_curve_path)

    # Evaluation Metrics (Accuracy, Sensitivity, Specificity, F1 Score)
    cm = confusion_matrix(true_labels, val_predicted_ids)
    metrics_bar_chart_path = os.path.join(weights_path, f"evaluation_metrics_fold_{fold}.png")
    save_evaluation_metrics(true_labels, val_predicted_ids, history.history, cm, metrics_bar_chart_path)

    # Save classification report as a text file
    classification_report_path = os.path.join(weights_path, f"classification_report_fold_{fold}.txt")
    save_classification_report(true_labels, val_predicted_ids, class_names, classification_report_path)


In [5]:
# # Define path to save best model
# weights_path = "../saved_models"
# os.makedirs(weights_path, exist_ok=True)

# best_val_f1score = -float('inf')  # Initialize best F1 score with a very low value

# train_metrics_df = []
# val_metrics_df = []

# # Set the number of epochs and folds
# EPOCHS = 1
# FOLDS = 5

# # K-fold Cross Validation
# kfold = KFold(n_splits=FOLDS, shuffle=True, random_state=42)
# acc_per_fold = []
# fold = 1

# # Define the base path for saving models
# save_dir = "../saved_models"
# os.makedirs(save_dir, exist_ok=True)

# # List to store accuracy results for comparison
# model_performance = []

# # Parameters
# NUM_MODELS = 1
# # NUM_EPOCHS = 14  # Or any number of epochs you prefer

# # TODO: Add function for custom configurations (eg. different optimizer, learning rate, etc.)

# # configurations that will be used in training
# configs = [
#     {"learning_rate": 0.001, "optimizer": "adam", "epochs": 14, "save_metrics": False},
#     # {"learning_rate": 0.0001, "optimizer": "adam", "epochs": 50, "save_metrics": False},
#     # {"learning_rate": 0.001, "optimizer": "sgd", "epochs": 50, "save_metrics": False},
#     # {"learning_rate": 0.0001, "optimizer": "sgd", "epochs": 50, "save_metrics": False},
# ]

# for i, config in enumerate(configs):
#     print(f"Training model {i + 1}/{len(configs)} with config: {config}")

#     # Training and validation loop for each epoch and fold
#     for epoch in range(EPOCHS):
#         print(f"\nEpoch {epoch+1}/{EPOCHS}")
#         fold = 1
#         fold_best_train_f1score = -float('inf')

#         for train_idx, val_idx in kfold.split(image_paths):
#             # Create subset datasets for training and validation
#             train_images, train_labels = image_paths[train_idx], labels[train_idx]
#             val_images, val_labels = image_paths[val_idx], labels[val_idx]

#             # Convert NumPy arrays back to TensorFlow datasets
#             train_ds = tf.data.Dataset.from_tensor_slices((train_images, train_labels))
#             val_ds = tf.data.Dataset.from_tensor_slices((val_images, val_labels))

#             # Normalize datasets and batch
#             normalization_layer = layers.Rescaling(1./255)
#             train_ds = train_ds.map(lambda x, y: (normalization_layer(x), y)).batch(batch_size)
#             val_ds = val_ds.map(lambda x, y: (normalization_layer(x), y)).batch(batch_size)

#             AUTOTUNE = tf.data.AUTOTUNE
#             train_ds = train_ds.cache().prefetch(buffer_size=AUTOTUNE)
#             val_ds = val_ds.cache().prefetch(buffer_size=AUTOTUNE)

#             # Create and compile model for each fold
#             model = create_model(num_classes, config)

#             # Train the model
#             if fold == FOLDS:
#                 # Validation loop
#                 val_predictions = model.predict(val_ds)
#                 avg_val_loss = model.evaluate(val_ds, verbose=0)[0]
#                 avg_val_accuracy, avg_val_precision, avg_val_recall, avg_val_f1, avg_val_auc = calculate_metrics(
#                     np.concatenate([y for x, y in val_ds]), val_predictions
#                 )

#                 print(f"\nValidation: \tFold {fold} - Loss: {avg_val_loss:.4f}, Accuracy: {avg_val_accuracy:.4f}, Precision: {avg_val_precision:.4f}, Recall: {avg_val_recall:.4f}, F1 Score: {avg_val_f1:.4f}, AUC Score: {avg_val_auc:.4f}")

#                 # Save the best model based on validation F1 score
#                 if avg_val_f1 > best_val_f1score:
#                     best_val_f1score = avg_val_f1
#                     val_metrics_dict = {"Loss": avg_val_loss, "Accuracy": avg_val_accuracy, "Precision": avg_val_precision, "Recall": avg_val_recall, "F1 Score": avg_val_f1, "AUC": avg_val_auc}
#                     val_metrics_df.append(val_metrics_dict)

#                     # Save model with best validation F1 score
#                     model.save(os.path.join(weights_path, f'mobilenetv2_best_f1score_epoch_{epoch+1}.h5'))
#                     print(f"Model with best F1 score during Validation saved at epoch {epoch + 1} with F1 Score of {best_val_f1score:.4f}")
#             else:
#                 # Training loop
#                 history = model.fit(train_ds, validation_data=val_ds, epochs=1, verbose=1)
#                 avg_train_loss = history.history['loss'][0]
#                 avg_train_accuracy, avg_train_precision, avg_train_recall, avg_train_f1, avg_train_auc = calculate_metrics(
#                     np.concatenate([y for x, y in train_ds]), model.predict(train_ds)
#                 )

#                 if avg_train_f1 > fold_best_train_f1score:
#                     fold_best_train_f1score = avg_train_f1

#                     print(f"Training: \tFold {fold} - Loss: {avg_train_loss:.4f}, Accuracy: {avg_train_accuracy:.4f}, Precision: {avg_train_precision:.4f}, Recall: {avg_train_recall:.4f}, F1 Score: {avg_train_f1:.4f}, AUC Score: {avg_train_auc:.4f}")
#                     train_metrics_dict = {"Loss": avg_train_loss, "Accuracy": avg_train_accuracy, "Precision": avg_train_precision, "Recall": avg_train_recall, "F1 Score": avg_train_f1, "AUC": avg_train_auc}
#                     train_metrics_df.append(train_metrics_dict)

#             # Reset fold metrics
#             fold += 1

#     # Save metrics after training
#     np.save(os.path.join(weights_path, 'train_metrics.npy'), train_metrics_df)
#     np.save(os.path.join(weights_path, 'val_metrics.npy'), val_metrics_df)


In [6]:
from tensorflow.keras.callbacks import EarlyStopping

# EarlyStopping callback configuration
early_stopping = EarlyStopping(
    monitor='val_loss',        # Monitor validation loss
    patience=3,                # Number of epochs with no improvement to stop training
    restore_best_weights=True  # Restore model weights from the epoch with the best value of the monitored metric
)


FOLDS = 5
# K-fold Cross Validation
kfold = KFold(n_splits=FOLDS, shuffle=True, random_state=42)
acc_per_fold = []
fold = 1
best_val_f1score = -float('inf')  # Initialize best F1 score with a very low value

# Define the base path for saving models
save_dir = "../saved_models"
os.makedirs(save_dir, exist_ok=True)

# List to store accuracy results for comparison
model_performance = []

train_metrics = []
val_metrics = []

# Parameters
NUM_MODELS = 1
# NUM_EPOCHS = 14  # Or any number of epochs you prefer

# TODO: Add function for custom configurations (eg. different optimizer, learning rate, etc.)

# configurations that will be used in training
configs = [
    {"learning_rate": 0.001, "optimizer": "adam", "epochs": 14, "save_metrics": False},
    # {"learning_rate": 0.0001, "optimizer": "adam", "epochs": 50, "save_metrics": False},
    # {"learning_rate": 0.001, "optimizer": "sgd", "epochs": 50, "save_metrics": False},
    # {"learning_rate": 0.0001, "optimizer": "sgd", "epochs": 50, "save_metrics": False},
]

for i, config in enumerate(configs):
    print(f"Training model {i + 1}/{len(configs)} with config: {config}")

    # Training and validation loop for each fold
    fold = 1
    for train_idx, val_idx in kfold.split(image_paths):
        print(f"\nFold {fold}/{FOLDS}...")

        # Create subset datasets for training and validation
        train_images, train_labels = image_paths[train_idx], labels[train_idx]
        val_images, val_labels = image_paths[val_idx], labels[val_idx]

        # Convert NumPy arrays back to TensorFlow datasets
        train_ds = tf.data.Dataset.from_tensor_slices((train_images, train_labels))
        val_ds = tf.data.Dataset.from_tensor_slices((val_images, val_labels))

        # Normalize datasets and batch
        normalization_layer = layers.Rescaling(1./255)
        train_ds = train_ds.map(lambda x, y: (normalization_layer(x), y)).batch(batch_size)
        val_ds = val_ds.map(lambda x, y: (normalization_layer(x), y)).batch(batch_size)

        AUTOTUNE = tf.data.AUTOTUNE
        train_ds = train_ds.cache().prefetch(buffer_size=AUTOTUNE)
        val_ds = val_ds.cache().prefetch(buffer_size=AUTOTUNE)

        # Create and compile model for each fold
        model = create_model(num_classes, config)

        # Train the model on the training set and validate on the current fold
        history = model.fit(train_ds, validation_data=val_ds, epochs=config["epochs"], callbacks=[early_stopping], verbose=1)

        # Evaluate on validation set after training
        val_predictions = model.predict(val_ds)
        avg_val_loss = model.evaluate(val_ds, verbose=0)[0]
        avg_val_accuracy, avg_val_precision, avg_val_recall, avg_val_f1, avg_val_auc = calculate_metrics(
            np.concatenate([y for x, y in val_ds]), val_predictions
        )

        print(f"\nValidation: \tFold {fold} - Loss: {avg_val_loss:.4f}, Accuracy: {avg_val_accuracy:.4f}, Precision: {avg_val_precision:.4f}, Recall: {avg_val_recall:.4f}, F1 Score: {avg_val_f1:.4f}, AUC Score: {avg_val_auc:.4f}")

        # Save the best model based on validation F1 score
        if avg_val_f1 > best_val_f1score:
            best_val_f1score = avg_val_f1
            model.save(os.path.join(save_dir, f'mobilenetv2_best_f1score_fold_{fold}.h5'))
            print(f"Model with best F1 score during Validation saved at Fold {fold} with F1 Score of {best_val_f1score:.4f}")

            # Save confusion matrix, loss curve, evaluation metrics for the best model
            save_best_model_visuals(history, model, val_ds, class_names, save_dir, fold)

        # Move to the next fold
        fold += 1

# Save metrics after training
np.save(os.path.join(save_dir, 'train_metrics.npy'), train_metrics)
np.save(os.path.join(save_dir, 'val_metrics.npy'), val_metrics)

Training model 1/1 with config: {'learning_rate': 0.001, 'optimizer': 'adam', 'epochs': 14, 'save_metrics': False}

Fold 1/5...
Epoch 1/14
80/80 ━━━━━━━━━━━━━━━━━━━━ 94s 1s/step - accuracy: 0.5930 - loss: 0.7429 - val_accuracy: 0.8232 - val_loss: 0.4209
Epoch 2/14
80/80 ━━━━━━━━━━━━━━━━━━━━ 62s 774ms/step - accuracy: 0.8134 - loss: 0.4129 - val_accuracy: 0.8576 - val_loss: 0.3515
Epoch 3/14
80/80 ━━━━━━━━━━━━━━━━━━━━ 69s 614ms/step - accuracy: 0.8618 - loss: 0.3312 - val_accuracy: 0.8670 - val_loss: 0.3159
Epoch 4/14
80/80 ━━━━━━━━━━━━━━━━━━━━ 53s 671ms/step - accuracy: 0.8926 - loss: 0.2840 - val_accuracy: 0.8732 - val_loss: 0.2930
Epoch 5/14
80/80 ━━━━━━━━━━━━━━━━━━━━ 74s 923ms/step - accuracy: 0.9041 - loss: 0.2524 - val_accuracy: 0.8905 - val_loss: 0.2766
Epoch 6/14
80/80 ━━━━━━━━━━━━━━━━━━━━ 60s 750ms/step - accuracy: 0.9119 - loss: 0.2290 - val_accuracy: 0.8936 - val_loss: 0.2642
Epoch 7/14
80/80 ━━━━━━━━━━━━━━━━━━━━ 63s 794ms/step - accuracy: 0.9216 - loss: 0.2108 - val_accuracy

NameError: name 'best_val_f1score' is not defined

## Testing

In [28]:
model.compile(
  optimizer=tfk.optimizers.Adam(),
  loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
  metrics=['acc'])


In [ ]:
NUM_EPOCHS = 5

# train model
trained_model = model.fit(train_ds,
                    validation_data=val_ds,
                    epochs=NUM_EPOCHS)

In [ ]:
predicted_batch = model.predict(image_batch)
predicted_id = tf.math.argmax(predicted_batch, axis=-1)
predicted_label_batch = class_names[predicted_id]
print(predicted_label_batch)

### Predictions

In [ ]:
plt.figure(figsize=(10,9))
plt.subplots_adjust(hspace=0.5)

for n in range(30):
  plt.subplot(6,5,n+1)
  plt.imshow(image_batch[n])
  plt.title(predicted_label_batch[n].title())
  plt.axis('off')
_ = plt.suptitle("Model predictions")